## Bokeh Setup

In [330]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

## Import data
source: https://transtats.bts.gov/DL_SelectFields.asp?gnoyr_VQ=FIL&QO_fu146_anzr=Nv4%20Pn44vr45

> This table contains domestic market data reported by U.S. air carriers, including carrier, origin, destination, and service class for enplaned passengers, freight and mail when both origin and destination airports are located within the boundaries of the United States and its territories.

In [331]:
import pandas as pd

domestic_carriers_df = pd.read_csv(
    "../data/1035327695_T_T100D_MARKET_US_CARRIER_ONLY.zip",
    compression="zip",
    dtype={
        "PASSENGERS": "int64",
        "FREIGHT": "int64",
        "MAIL": "int64",
        "DISTANCE": "int64",
        }
    )
domestic_carriers_df.drop("Unnamed: 22", axis=1, inplace=True)  # TBD ?!
domestic_carriers_df.head()

,PASSENGERS,FREIGHT,MAIL,DISTANCE,UNIQUE_CARRIER,UNIQUE_CARRIER_NAME,CARRIER,CARRIER_NAME,CARRIER_GROUP_NEW,ORIGIN_AIRPORT_ID,...,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,DEST_AIRPORT_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_NM,YEAR,QUARTER,MONTH
0,0,0,0,1180,AMQ,Ameristar Air Cargo,AMQ,Ameristar Air Cargo,4,10171,...,TX,Texas,14711,SCE,"State College, PA",PA,Pennsylvania,2021,1,1
1,0,0,0,166,AMQ,Ameristar Air Cargo,AMQ,Ameristar Air Cargo,4,11049,...,TX,Texas,10171,ADS,"Dallas, TX",TX,Texas,2021,1,1
2,0,0,0,751,AMQ,Ameristar Air Cargo,AMQ,Ameristar Air Cargo,4,12878,...,IN,Indiana,10171,ADS,"Dallas, TX",TX,Texas,2021,1,1
3,0,0,0,1073,AMQ,Ameristar Air Cargo,AMQ,Ameristar Air Cargo,4,16091,...,MI,Michigan,11049,CLL,"College Station/Bryan, TX",TX,Texas,2021,1,1
4,0,0,0,563,AMQ,Ameristar Air Cargo,AMQ,Ameristar Air Cargo,4,10171,...,TX,Texas,11540,ELP,"El Paso, TX",TX,Texas,2021,1,1


In [332]:
domestic_carriers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176531 entries, 0 to 176530
Data columns (total 22 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   PASSENGERS           176531 non-null  int64 
 1   FREIGHT              176531 non-null  int64 
 2   MAIL                 176531 non-null  int64 
 3   DISTANCE             176531 non-null  int64 
 4   UNIQUE_CARRIER       176531 non-null  object
 5   UNIQUE_CARRIER_NAME  176531 non-null  object
 6   CARRIER              176531 non-null  object
 7   CARRIER_NAME         176531 non-null  object
 8   CARRIER_GROUP_NEW    176531 non-null  int64 
 9   ORIGIN_AIRPORT_ID    176531 non-null  int64 
 10  ORIGIN               176531 non-null  object
 11  ORIGIN_CITY_NAME     176531 non-null  object
 12  ORIGIN_STATE_ABR     176531 non-null  object
 13  ORIGIN_STATE_NM      176531 non-null  object
 14  DEST_AIRPORT_ID      176531 non-null  int64 
 15  DEST                 176531 non-nu

## Set constants (to be dynamically set by widgets in the future)

In [333]:
MEASUREMENTS = ["passengers", "freight", "mail"]
MEASUREMENT = MEASUREMENTS[0]  # So far, MEASUREMENT can only have a single value
NUMBER_OF_AIRLINES_TO_CONSIDER = 10

## Find N largest carriers by measurement


In [334]:
biggest_carriers = domestic_carriers_df.groupby(['UNIQUE_CARRIER_NAME'])[MEASUREMENT.upper()].sum().reset_index()
biggest_carriers = biggest_carriers.sort_values(by=MEASUREMENT.upper(), ascending=False)[:NUMBER_OF_AIRLINES_TO_CONSIDER]
biggest_carriers["POSITION"] = range(1, len(biggest_carriers) + 1)  # add position column for tooltip
biggest_carriers.reset_index(drop=True, inplace=True)
biggest_carriers

,UNIQUE_CARRIER_NAME,PASSENGERS,POSITION
0,Southwest Airlines Co.,85454969,1
1,American Airlines Inc.,68955973,2
2,Delta Air Lines Inc.,64057021,3
3,United Air Lines Inc.,39160902,4
4,SkyWest Airlines Inc.,24778440,5
5,Spirit Air Lines,19973906,6
6,JetBlue Airways,16188917,7
7,Alaska Airlines Inc.,15326139,8
8,Frontier Airlines Inc.,14068241,9
9,Republic Airline,13085272,10


## Plot the N largest carriers by measurement

In [335]:
from bokeh.plotting import figure, show
from bokeh.models import NumeralTickFormatter

plot_title = f"Top {NUMBER_OF_AIRLINES_TO_CONSIDER} carriers by domestic {MEASUREMENT}"

TOOLTIPS = [
    ("Index", "$index"),  # TBD remove, use position instead. OR: display index + 1
    ("Position", "@POSITION"),  # TBD, use data from POSITION column
    ("Carrier", "@x"),
    (MEASUREMENT.capitalize(), "@PASSENGERS"),  # TBD, use MEASUREMENT instead
]

##############
# An example of what a tooltip should look like:
#
# Position: 1
# Carrier: United Parcel Service
# Mail: 353,580,883
#
##############

p = figure(x_range=biggest_carriers["UNIQUE_CARRIER_NAME"], title=plot_title, height=300, sizing_mode="stretch_width", tooltips=TOOLTIPS)
p.vbar(x=biggest_carriers["UNIQUE_CARRIER_NAME"], top=biggest_carriers[MEASUREMENT.upper()], legend_label=MEASUREMENT.capitalize(), width=0.6)

p.xgrid.grid_line_color = None
p.yaxis.formatter = NumeralTickFormatter(format="0,0")
# from math import pi
# p.xaxis.major_label_orientation = pi/4  # rotate labels

show(p)

## Calculate total number of passengers, freight, and mail for each month for N largest carriers

In [336]:

# Create dataframe with one line per month. Each line contains the sums of passengers, freight, and mail for each month for N largest carriers for that month.

dimensions = [measurement.upper() for measurement in MEASUREMENTS]  # create list of dimensions to create sums for

df_monthly = pd.DataFrame()
for carrier in biggest_carriers["UNIQUE_CARRIER_NAME"]:
    df = domestic_carriers_df[domestic_carriers_df["UNIQUE_CARRIER_NAME"] == carrier].groupby(['MONTH'])[dimensions].sum()
    df["UNIQUE_CARRIER_NAME"] = carrier
    df_monthly = df_monthly.append(df)

df_monthly = df_monthly.groupby(['MONTH']).sum()
df_monthly

,PASSENGERS,FREIGHT,MAIL
MONTH,,,
1,20423213,76416978,46710643
2,20486428,81595930,37970725
3,33085774,99807707,44170148
4,36990012,95730688,46005655
5,44812849,102722167,44660510
6,50836927,103738042,41593383
7,56515483,103537892,45956987
8,51471440,102781501,46324061
9,46427654,103944129,35114242


## Plot development of total number of passengers, freight, and mail over time for N largest carriers
with interactive legend to mute glyphs

In [342]:
from bokeh.palettes import Category10
from bokeh.plotting import figure, show
from bokeh.models import NumeralTickFormatter

plot_title = f"Development of domestic passengers, freight, and mail for the top {NUMBER_OF_AIRLINES_TO_CONSIDER} carriers"

TOOLTIPS = [
    ("Month", "$index"),
    ("Amount", "@y{(0,0)}"),
]

p = figure(title=plot_title, height=300, sizing_mode="stretch_width", tooltips=TOOLTIPS)

color = 0
for measurement in MEASUREMENTS:
    p.line(df_monthly.index, df_monthly[measurement.upper()], legend_label=measurement.capitalize(), width=2, color=Category10[3][color], alpha=1, muted_alpha=0.2)
    color += 1

p.yaxis.formatter = NumeralTickFormatter(format="0,0")
p.xaxis.axis_label = "Month"  # TBD: x axis ticks display months, optimally month names
p.legend.click_policy="mute"

show(p)